# Huge Tensor Bug - Data loss

## Requirements
I'm running this on the standard 0.7
[RAPIDS docker containers](https://hub.docker.com/r/rapidsai/rapidsai) and also
need the following `pip` dependencies installed:

In [26]:
!pip install torch==1.0.1

If you're running this on your local machine you should have most things installed

## CODE

### Imports

In [1]:
import torch

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

#### In order to demonstrate this bug we need a large tensor

In [4]:
features = torch.LongTensor(15000000, 45).random_(0, 2**22)
#features = torch.FloatTensor(15000000, 90).random_(-1, 1)

In [5]:
features[0]

tensor([2874842, 1426886, 2900003, 1340271, 1255904,   53916,  150306, 3135209,
        1566565, 3700984,  199050, 2326381,  436634,    1117, 1546400, 4001063,
        1472543, 1806464,  201497,  716067,   94419,  212859,  543911,  908213,
        4174394, 3005976, 2171144, 2967139,  813713, 1516383,  686261, 3008736,
        1182924, 3776220, 3716994, 3882399, 4064310, 2592183, 3436898, 1882412,
         546035, 3549929, 4055420, 1432250, 1471965])

In [6]:
features[14999999]

tensor([2167864, 3165248, 1245852, 2146404, 3948645,  265299,  844671, 3354802,
         590113, 2841125,  958101, 1508428, 3405411, 1697418,  301883, 3097217,
         620701,  818595, 3024319, 3623090, 1033113,  289799,  200632, 3951973,
        2293208, 1706245, 1716449, 3183350, 2641135, 1216988, 1998064, 2379344,
        4049610, 3027779, 1925729, 3419792, 2398803, 3161385, 2171724, 2212434,
        2186084, 3258908, 1923136, 3271952, 3799253])

Validate that the last element in the dataframe has data

In [7]:
features2 = features.cuda()

In [8]:
features2[0]

tensor([2874842, 1426886, 2900003, 1340271, 1255904,   53916,  150306, 3135209,
        1566565, 3700984,  199050, 2326381,  436634,    1117, 1546400, 4001063,
        1472543, 1806464,  201497,  716067,   94419,  212859,  543911,  908213,
        4174394, 3005976, 2171144, 2967139,  813713, 1516383,  686261, 3008736,
        1182924, 3776220, 3716994, 3882399, 4064310, 2592183, 3436898, 1882412,
         546035, 3549929, 4055420, 1432250, 1471965], device='cuda:0')

In [9]:
features2[14999999]

tensor([2167864, 3165248, 1245852, 2146404, 3948645,  265299,  844671, 3354802,
         590113, 2841125,  958101, 1508428, 3405411, 1697418,  301883, 3097217,
         620701,  818595, 3024319, 3623090, 1033113,  289799,  200632, 3951973,
        2293208, 1706245, 1716449, 3183350, 2641135, 1216988, 1998064, 2379344,
        4049610, 3027779, 1925729, 3419792, 2398803, 3161385, 2171724, 2212434,
        2186084, 3258908, 1923136, 3271952, 3799253], device='cuda:0')

In [10]:
features3 = features2[torch.randperm(len(features2))]

In [11]:
features3[0]

tensor([ 232678, 4128972,   14863, 2813414, 2908796, 3753613, 2405223, 2602856,
        3152401,  829190, 1368736, 4086470, 1045409, 3881271, 2189020, 1881501,
        3215912, 2919082, 3253875, 3140631,  600729, 3710989, 3068196, 1799416,
        4106247, 2957127,  479433, 1466325, 2538156,  754164, 1337687, 1193884,
        1258663, 2828806, 3600707,   32696, 3391579, 3882006, 1153262,  548138,
        1612788, 2112974, 3762296,  572073, 1035993], device='cuda:0')

In [12]:
features3[14999999]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')

Here's the boundary where this issue comes into play for this tensor.  All data after this point is 0.

In [13]:
features3[11930460:11930470]

tensor([[3970338, 3882211, 2257313, 2996034, 2618937,  821899,  127752, 1941789,
         2321753, 2531483, 1901012,   88791, 3273090,  396874, 1220971,  210458,
         3162683, 1267541,  601566, 4155051,  320741, 1119626, 3992337, 1943076,
         2601893, 2984709,  551029, 1772495, 3970556, 1795816,  788027, 3360688,
         1369740, 2540040, 2186604, 2789128, 1974345, 1313554,  300582, 2928085,
         3571232,  401539, 1025006, 1408648,   39732],
        [3751935, 1031756, 2405813, 3240593,  170331,  633957, 1879662, 1310566,
         2340410, 2586015, 1142524, 1461920, 3627306, 2920978,  707182, 2123344,
         1924881, 2377270, 3865196,  376196, 3836740, 1428437, 1953505, 1174601,
          383992, 4049732,  537351,  535581,  445718, 3649950, 1464503, 3661558,
         2528252, 4129869, 1947744, 1848788,  993754, 1634504, 2315119,  298398,
         2578542, 3066868, 3357185,  601327,  617268],
        [2204282, 3078495,  426366,  225245, 2141891,  191852, 2205630, 1429943,